In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [2]:
df = pd.read_csv('data_processing/landmark2.csv')
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)
print(keypoint_dataset.__len__())

2999


In [3]:
import model as m
model = m.SimpleNN().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Model's state_dict:
fc1.weight 	 torch.Size([20, 63])
fc1.bias 	 torch.Size([20])
fc2.weight 	 torch.Size([10, 20])
fc2.bias 	 torch.Size([10])
fc3.weight 	 torch.Size([3, 10])
fc3.bias 	 torch.Size([3])


In [4]:
num_epochs = 1000
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), "models/model3.pth")

Epoch [1/1000], Loss: 1.0866
Epoch [2/1000], Loss: 1.0753
Epoch [3/1000], Loss: 1.0535
Epoch [4/1000], Loss: 1.0054
Epoch [5/1000], Loss: 0.9547
Epoch [6/1000], Loss: 0.9010
Epoch [7/1000], Loss: 0.9242
Epoch [8/1000], Loss: 0.8136
Epoch [9/1000], Loss: 0.8095
Epoch [10/1000], Loss: 0.7969
Epoch [11/1000], Loss: 0.7399
Epoch [12/1000], Loss: 0.7537
Epoch [13/1000], Loss: 0.7721
Epoch [14/1000], Loss: 0.7738
Epoch [15/1000], Loss: 0.7099
Epoch [16/1000], Loss: 0.6991
Epoch [17/1000], Loss: 0.6617
Epoch [18/1000], Loss: 0.6619
Epoch [19/1000], Loss: 0.6815
Epoch [20/1000], Loss: 0.7439
Epoch [21/1000], Loss: 0.6747
Epoch [22/1000], Loss: 0.6783
Epoch [23/1000], Loss: 0.6131
Epoch [24/1000], Loss: 0.6716
Epoch [25/1000], Loss: 0.7094
Epoch [26/1000], Loss: 0.7454
Epoch [27/1000], Loss: 0.6514
Epoch [28/1000], Loss: 0.6471
Epoch [29/1000], Loss: 0.6520
Epoch [30/1000], Loss: 0.6686
Epoch [31/1000], Loss: 0.6754
Epoch [32/1000], Loss: 0.7043
Epoch [33/1000], Loss: 0.6875
Epoch [34/1000], Lo